In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import seaborn as sns
import re


# Import des données des bilans alimentaires pour l'an 2012, 2013

# Bilans Alimentaires> [Pays]="Tout Sélectionner"+ [Elements]="Tout Sélectionner"+ 
#                      [Produits]="Produits Vegetaux > (Liste)"+ [Années]=2013 ,2012  -> dans veg.csv
# options (Table,Virgule,Code)

# Bilans Alimentaires> [Pays]="Tout Sélectionner"+ [Elements]="Tout Sélectionner"+ 
#                      [Produits]="Produits Animaux > (Liste)"+ [Années]=2013 ,2012  -> dans ani.csv
# options (Table,Virgule,Code)

veg = pd.read_csv("veg.csv")   # produits issus des végétaux
ani = pd.read_csv("ani.csv")   # produits issus des animaux


# Ajout d'une colonne supplémentaire 'origin' du produit
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

# Suppression des colonnes inutiles
temp = temp.drop(['Code Domaine', 'Domaine', 'Code Élément', 'Code Année'], 1)
# Renommage des colonnes restantes
temp.columns = ['Code Pays', 'Pays', 'Element', 'Code Produit', 'Produit', 'An', 'Valeur', 'origin']

# Suprimer les lignes ou Valeur = NaN, 0.000
temp = temp.fillna(0)            # Remplace les NaN par des 0
temp = temp[temp.Valeur != 0]    # Ne conserve que les lignes ou 'Valeur' != 0

# Suppression des lignes relatives aux quatres régions de la Chine :
#    (Chine - RAS de Hong-Kong) + (Chine - RAS de Macao) + (Chine, continentale ) + (Chine, Taiwan Province de)
# Ne restent que les lignes relatives à la 'Chine'
temp = temp[temp.Pays != 'Chine - RAS de Hong-Kong']
temp = temp[temp.Pays != 'Chine - RAS de Macao']
temp = temp[temp.Pays != 'Chine, continentale']
temp = temp[temp.Pays != 'Chine, Taiwan Province de']


# Constitution du Dataframe et du fichier csv des Pays en sous-nutrition
secu_alimentaire = pd.read_csv("secu_alimentaire.csv")

# Elimination des colonnes inutiles
secu = secu_alimentaire.drop(['Code Domaine', 'Domaine', 'Code Élément', 'Élément', 'Code Produit', 'Code année', 'Unité'], 1)

# Renommage des colonnes restantes
secu.columns = ['Code pays','Pays', 'Produit', 'An', 'Valeur']

# Filtrage
secu = secu[secu.Produit == 'Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)']
secu = secu[(secu.An == '2011-2013') | (secu.An == '2012-2014') | (secu.An == '2013-2015') | \
            (secu.An == '2014-2016') | (secu.An == '2015-2017')]

# Formatage approprié de l'année
secu.loc[secu['An'] == '2011-2013', 'An'] = 2012
secu.loc[secu['An'] == '2012-2014', 'An'] = 2013
secu.loc[secu['An'] == '2013-2015', 'An'] = 2014
secu.loc[secu['An'] == '2014-2016', 'An'] = 2015
secu.loc[secu['An'] == '2015-2017', 'An'] = 2016

# Suppression des lignes relatives aux quatres régions de la Chine :
#    (Chine - RAS de Hong-Kong) + (Chine - RAS de Macao) + (Chine, continentale ) + (Chine, Taiwan Province de)
# Ne restent que les lignes relatives à la 'Chine'
secu = secu[secu.Pays != 'Chine - RAS de Hong-Kong']
secu = secu[secu.Pays != 'Chine - RAS de Macao']
secu = secu[secu.Pays != 'Chine, continentale']
secu = secu[secu.Pays != 'Chine, Taiwan Province de']

# Elimination du caractère '<' lorsqu'il apparaît dans la colonne 'Valeur', ex : '<0.6'
secu['Valeur'] = secu['Valeur'].replace(to_replace='[<]+', value='', regex=True) 

secu = secu.fillna(0)            # Remplace les NaN par des 0
secu = secu[secu.Valeur != 0]    # Ne conserve que les lignes ou 'Valeur' != 0

# Passer la colonne 'Valeur' de str à int, car il y a un sum() à faire par la suite
secu.Valeur = pd.to_numeric(secu.Valeur, errors='coerce')

secu = secu.pivot_table(index=["Code pays", "Pays",  "An"],  columns = ["Produit"], values=["Valeur"], aggfunc=sum)
secu = secu.reset_index()
secu.columns = secu.columns.droplevel(level=[0])
secu.columns = ['Code pays', 'Pays', 'An', 'nb_sous_aliment']  # nb_sous_aliment en Millions personnes  (moyenne sur 3ans)

# Pour l'import dans la table  'sous_nutrition'  de la base de données Mysql
secu.to_csv("sous_nutrition.csv", index = False)

In [2]:
##   ----- QUESTION 1 -----

#     Nombre total d’humains sur la planète (2013) 
# 
# Bilans Alimentaires> [Pays]="Tout Sélectionner"+ [Elements]="Population totale"+ [Produits]="Population"+ [Années]=2013 ,2012
#  avec options (Table,Virgule,Code) ->   -> 8413993000 -> erreur la Chine est compté 2 fois

# Bilans Alimentaires> [Régions]="Monde>(Liste) "+ [Elements]="Population totale"+ [Produits]="Population"+ [Années]=2013 ,2012
#  avec options (Table,Virgule,Code) ->   -> 6997326000

pop = pd.read_csv("pop.csv", sep=",")

# Suppression des colonnes inutiles
pop = pop.drop(['Code Domaine', 'Domaine', 'Code Élément', 'Élément', 'Code Produit', 'Produit', 'Code Année'],1)

# Renommage des colonnes restantes
pop.columns = ['Code Pays', 'Pays', 'An', 'Population']

# Pour éviter de compter 2 fois le nombre d'habitants de la Chine (1416667000). Ne reste que la 'Chine'
pop = pop[pop.Pays != 'Chine - RAS de Hong-Kong']
pop = pop[pop.Pays != 'Chine - RAS de Macao']
pop = pop[pop.Pays != 'Chine, continentale']
pop = pop[pop.Pays != 'Chine, Taiwan Province de']
# Chine   = (Chine - RAS de Hong-Kong) + (Chine - RAS de Macao) + (Chine, continentale ) + (Chine, Taiwan Province de)
# 1416667 = 7204000                    + 566000                 + 1385567000             + 23330000

# *1000 car la Population est exprimée en '1000 personnes'
NbHumainTotal = pop[pop['An'] == 2013]['Population'].sum() * 1000
print("Nombre total d'humains sur la planète en 2013 = ", NbHumainTotal)

# Merge sur le colonnes ('Pays','An') pour ajouter la colonne population à 'temp'
temp = pd.merge(pop, temp)

# Pour l'import dans la table  'population'  de la base de données Mysql
pop.to_csv("population.csv", index = False)


Nombre total d'humains sur la planète en 2013 =  6997326000


In [3]:
#   ----- QUESTION 2 -----

# Exemple du Blé(2511) en 2013 en France(68)
temp[(temp['Code Produit'] == 2511) & (temp['An'] == 2013) & (temp['Code Pays'] == 68)]

# Disponibilité int. = (Production Quantité)+(Importations Quantité)+(Variation de stock) -   Exportations Quantité
# Disponibilité int. = Nourriture +   (Aliments pour animaux)+(Semences)+(Pertes)+(Traitement)+(Autres Utilisations)

# 'Valeur'      - 'Milliers de tonnes'
# 'Population'  - '1000 personnes'

,Code Pays,Pays,An,Population,Element,Code Produit,Produit,Valeur,origin
62365,68,France,2013,64291,Production,2511,Blé,38614.00,vegetal
62366,68,France,2013,64291,Importations - Quantité,2511,Blé,2055.00,vegetal
62367,68,France,2013,64291,Variation de stock,2511,Blé,1131.00,vegetal
62368,68,France,2013,64291,Exportations - Quantité,2511,Blé,21502.00,vegetal
62369,68,France,2013,64291,Disponibilité intérieure,2511,Blé,20298.00,vegetal
62370,68,France,2013,64291,Aliments pour animaux,2511,Blé,7822.00,vegetal
62371,68,France,2013,64291,Semences,2511,Blé,748.00,vegetal
62372,68,France,2013,64291,Pertes,2511,Blé,358.00,vegetal
62373,68,France,2013,64291,Traitement,2511,Blé,1575.00,vegetal
62374,68,France,2013,64291,Autres Utilisations,2511,Blé,2824.00,vegetal


In [4]:
#   ----- QUESTION 3 -----

# Réorganisation de la dataframe pour présenter les 'Valeur' en fonction du Pays/Produit/Année/Element
data = temp.pivot_table(index=["Code Pays", "Pays", "Code Produit", "Produit", "An", "origin", "Population"],
                        columns = ["Element"], values=["Valeur"], aggfunc=sum)
data = data.reset_index()

# Mis à plat du multiIndex
data.columns = data.columns.droplevel(level=[0])
data.columns = ['Code Pays', 'Pays', 'Code Produit', 'Produit', 'An', 'origin', 'Population', 'Aliments pour animaux', 'Autres Utilisations', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 'Semences', 'Traitement', 'Variation de stock']


# Attention ! la fonction pivot_table() crée automatiquement des NaN à la place des combinaisons manquante, 
#   qu'il faut éliminer à cause de possibles calculs ultérieurs
data = data.fillna(0)            # Remplace les NaN par des 0



# Pour l'import dans la table 'dispo_alim'  de la base de données Mysql
dispo_alim = data.drop(['Population', 'Aliments pour animaux', 'Autres Utilisations',
                'Disponibilité alimentaire en quantité (kg/personne/an)', 
                'Disponibilité intérieure', 'Exportations - Quantité', 
                'Importations - Quantité', 'Pertes', 'Production', 'Semences', 'Traitement', 'Variation de stock'], 1)
# Alignement des colonnes conforme à la structure de la table SQL dispo_alim
dispo_alim = dispo_alim[['Code Pays', 'Pays', 'An', 'Produit', 'Code Produit', 'origin', 'Nourriture', 
                      'Disponibilité alimentaire (Kcal/personne/jour)',
                      'Disponibilité de protéines en quantité (g/personne/jour)',
                      'Disponibilité de matière grasse en quantité (g/personne/jour)']]
dispo_alim.to_csv("dispo_alim.csv", index = False)


# Pour l'import dans la table  'equilibre_prod'  de la base de données Mysql
equilibre_prod = data.drop(['Population', 'origin',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',                           
       'Importations - Quantité', 'Production', 'Variation de stock'], 1)
# Alignement des colonnes conforme à la structure de la table SQL equilibre_prod
equilibre_prod = equilibre_prod[['Code Pays', 'Pays', 'An', 'Produit', 'Code Produit', 'Disponibilité intérieure',
                                 'Aliments pour animaux', 'Semences', 'Pertes', 'Traitement',
                                 'Nourriture', 'Autres Utilisations']]
equilibre_prod.to_csv("equilibre_prod.csv", index = False)



# (*1000 car exprimé en Milliers de personnes) - (*365 pour 1 an)
data.loc[data['Disponibilité alimentaire (Kcal/personne/jour)'] != 0,'dispo_k_cal'] = data['Disponibilité alimentaire (Kcal/personne/jour)'] * data['Population'] * 1000 * 365

#  (*1000 car exprimé en Milliers de personnes) - (/1000 pour avoir des kg de proteines)
data.loc[data['Disponibilité de protéines en quantité (g/personne/jour)'] != 0,'dispo_kg_prot'] = data['Disponibilité de protéines en quantité (g/personne/jour)'] * data['Population'] * 365

# Pour mieux présenter les 2 nouvelles colonnes 'dispo_k_cal'et, 'dispo_kg_prot'
data = data.reindex(columns = ['Code Pays', 'Pays', 'Code Produit', 'Produit', 'An', 'dispo_k_cal', 'dispo_kg_prot', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'origin', 'Population', 'Aliments pour animaux', 'Autres Utilisations', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 'Semences', 'Traitement', 'Variation de stock'])

data.head(100)

,Code Pays,Pays,Code Produit,Produit,An,dispo_k_cal,dispo_kg_prot,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),origin,...,Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,1,Arménie,2511,Blé,2012,1.078267e+12,32174607.65,995.0,29.69,vegetal,...,3.53,532.0,1.0,496.0,377.0,29.0,243.0,23.0,10.0,-207.0
1,1,Arménie,2511,Blé,2013,1.112684e+12,33163184.60,1024.0,30.52,vegetal,...,3.60,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0
2,1,Arménie,2513,Orge,2012,1.083685e+09,32510.55,1.0,0.03,vegetal,...,0.01,172.0,0.0,1.0,1.0,14.0,170.0,14.0,7.0,0.0
3,1,Arménie,2513,Orge,2013,NaN,NaN,0.0,0.00,vegetal,...,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0
4,1,Arménie,2514,Maïs,2012,NaN,10836.85,0.0,0.01,vegetal,...,0.00,58.0,0.0,39.0,0.0,5.0,19.0,0.0,0.0,0.0
5,1,Arménie,2514,Maïs,2013,NaN,10866.05,0.0,0.01,vegetal,...,0.00,102.0,0.0,82.0,0.0,7.0,21.0,0.0,0.0,0.0
6,1,Arménie,2515,Seigle,2012,1.083685e+09,32510.55,1.0,0.03,vegetal,...,0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,1,Arménie,2515,Seigle,2013,1.086605e+09,21732.10,1.0,0.02,vegetal,...,0.00,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,1,Arménie,2516,Avoine,2012,2.167370e+09,86694.80,2.0,0.08,vegetal,...,0.03,4.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0
9,1,Arménie,2516,Avoine,2013,2.173210e+09,97794.45,2.0,0.09,vegetal,...,0.03,6.0,0.0,1.0,1.0,0.0,5.0,0.0,0.0,0.0


In [5]:
#   ----- QUESTION 4 -----

data = data.fillna(0)            # Remplace les NaN par des 0

# Ratio "énergie/poids" en kcal/kg, par /pays/produit/An
data.loc[data['Disponibilité alimentaire en quantité (kg/personne/an)'] != 0,'ratio_kcal_kg'] = data['dispo_k_cal'] / (data['Disponibilité alimentaire en quantité (kg/personne/an)'] * data['Population']*1000)

# Ratio "poids de protéines/poids total" en %de poids, par pays/produit/An
data.loc[data['Disponibilité alimentaire en quantité (kg/personne/an)'] != 0,'ratio_prot'] = data['dispo_kg_prot'] / (data['Disponibilité alimentaire en quantité (kg/personne/an)'] * data['Population']*1000)

# Pour mieux présenter les 2 nouvelles colonnes 'ratio_kcal_kg' et 'ratio_prot'
data = data.reindex(columns = ['Code Pays', 'Pays', 'Code Produit', 'Produit', 'An', 'ratio_kcal_kg', 'ratio_prot', 'dispo_k_cal', 'dispo_kg_prot', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'origin', 'Population', 'Aliments pour animaux', 'Autres Utilisations', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 'Semences', 'Traitement', 'Variation de stock'])
data.head(100)


,Code Pays,Pays,Code Produit,Produit,An,ratio_kcal_kg,ratio_prot,dispo_k_cal,dispo_kg_prot,Disponibilité alimentaire (Kcal/personne/jour),...,Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,1,Arménie,2511,Blé,2012,2860.997322,0.085370,1.078267e+12,32174607.65,995.0,...,3.53,532.0,1.0,496.0,377.0,29.0,243.0,23.0,10.0,-207.0
1,1,Arménie,2511,Blé,2013,2861.868300,0.085297,1.112684e+12,33163184.60,1024.0,...,3.60,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0
2,1,Arménie,2513,Orge,2012,2147.058824,0.064412,1.083685e+09,32510.55,1.0,...,0.01,172.0,0.0,1.0,1.0,14.0,170.0,14.0,7.0,0.0
3,1,Arménie,2513,Orge,2013,NaN,NaN,0.000000e+00,0.00,0.0,...,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0
4,1,Arménie,2514,Maïs,2012,0.000000,0.121667,0.000000e+00,10836.85,0.0,...,0.00,58.0,0.0,39.0,0.0,5.0,19.0,0.0,0.0,0.0
5,1,Arménie,2514,Maïs,2013,0.000000,0.121667,0.000000e+00,10866.05,0.0,...,0.00,102.0,0.0,82.0,0.0,7.0,21.0,0.0,0.0,0.0
6,1,Arménie,2515,Seigle,2012,2433.333333,0.073000,1.083685e+09,32510.55,1.0,...,0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,1,Arménie,2515,Seigle,2013,3041.666667,0.060833,1.086605e+09,21732.10,1.0,...,0.00,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,1,Arménie,2516,Avoine,2012,2281.250000,0.091250,2.167370e+09,86694.80,2.0,...,0.03,4.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0
9,1,Arménie,2516,Avoine,2013,1972.972973,0.088784,2.173210e+09,97794.45,2.0,...,0.03,6.0,0.0,1.0,1.0,0.0,5.0,0.0,0.0,0.0


In [6]:
#   ----- QUESTION 5 -----

# --- 5 aliments parmi les 20 aliments les plus caloriques (ratio énergie/poids = kcal/kg )

data5 = data[(data.ratio_kcal_kg != 0) & (data.An == 2013)]

# Moyenne du ratio 'ratio_kcal_kg' par produit tout pays confondu
data5.groupby(['Produit','An','Code Produit']).mean().sort_values(by = 'ratio_kcal_kg', ascending = False)

# 'Huiles de Foie de Poissons' > 'Huile de Sésame' > 'Huile de Son de Riz' > 'Huile d'Olive' >  'Huile de Germe de Maïs' >
# 'Huile de Colza&Moutarde' > 'Huile d'Arachide' > 'Huile de Tournesol' > 'Huile de Palmistes' > 'Huile Graines de Coton


,,,Code Pays,ratio_kcal_kg,ratio_prot,dispo_k_cal,dispo_kg_prot,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Population,Aliments pour animaux,Autres Utilisations,...,Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Produit,An,Code Produit,,,,,,,,,,,,,,,,,,,,,
Huiles de Foie de Poisso,2013,2782,148.130435,11809.003531,0.000000e+00,4.351498e+09,0.000000e+00,1.478261,0.000000,103498.173913,0.173913,0.086957,...,0.158261,0.739130,0.826087,1.000000,0.478261,0.000000,0.608696,0.000000,0.000000,0.000000
Huile de Sésame,2013,2579,125.960526,10192.188616,0.000000e+00,8.442837e+10,0.000000e+00,4.539474,0.000000,71320.236842,0.000000,5.144737,...,0.504868,14.565789,1.065789,1.473684,9.368421,0.000000,14.157895,0.000000,0.013158,0.000000
Huile de Son de Riz,2013,2581,134.500000,9314.765053,5.738994e-04,5.091273e+11,3.314826e+04,6.785714,0.000714,252538.500000,0.000000,17.000000,...,0.755000,77.142857,2.142857,2.214286,60.142857,0.000000,79.428571,0.000000,0.000000,-2.357143
Huile d'Olive,2013,2580,127.141791,9301.823385,0.000000e+00,1.831233e+11,0.000000e+00,19.641791,0.000000,32658.850746,0.007463,2.335821,...,2.222090,22.880597,13.753731,14.029851,20.552239,0.000000,26.447761,0.000000,0.000000,-3.820896
Huile de Germe de Maïs,2013,2582,122.000000,9278.454590,1.511230e-04,1.740905e+11,2.852475e+03,14.803571,0.001339,50579.276786,0.000000,0.660714,...,1.669286,20.312500,7.848214,6.607143,19.607143,0.044643,20.991071,0.000000,0.000000,0.571429
Huile de Colza&Moutarde,2013,2574,128.821782,9157.170010,9.184470e-04,8.483826e+11,5.696790e+04,39.891089,0.005545,59970.742574,6.930693,127.742574,...,4.509802,231.762376,73.108911,66.584158,97.059406,0.029703,237.712871,0.000000,0.009901,0.613861
Huile d'Arachide,2013,2572,130.384615,9118.384270,1.090203e-04,3.243785e+11,3.897077e+02,19.576923,0.000096,53901.538462,0.000000,11.076923,...,2.213846,47.788462,2.038462,2.182692,36.913462,0.009615,47.980769,0.000000,0.000000,-0.307692
Huile de Tournesol,2013,2573,125.330935,9041.569959,3.968355e-04,6.456373e+11,6.191080e+04,66.805755,0.006187,45553.827338,0.000000,23.532374,...,7.551439,96.928058,65.129496,62.381295,73.338129,0.057554,98.294964,0.000000,0.007194,1.395683
Huile de Palmistes,2013,2576,136.478261,9010.065306,5.862816e-04,1.855002e+11,9.402598e+03,8.728261,0.000543,68599.260870,0.000000,50.271739,...,0.991304,71.402174,35.260870,34.597826,20.913043,0.282609,71.782609,0.000000,0.000000,0.369565


In [7]:
#   ----- QUESTION 5 -----

# --- 5 aliments parmi les 20 aliments les plus riches en proteines

data5 = data[(data.ratio_prot != 0) & (data.An == 2013)]

# Moyenne du ratio 'ratio_prot' par produit tout pays confondu
data5.groupby(['Produit','An','Code Produit']).mean().sort_values(by = 'ratio_prot', ascending = False)

# 'Graines Colza/Moutarde' > 'Soja' 'Arachides Decortiquees' > 'Pois' > 'Légumineuses Autres' > 'Haricots'


,,,Code Pays,ratio_kcal_kg,ratio_prot,dispo_k_cal,dispo_kg_prot,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Population,Aliments pour animaux,Autres Utilisations,...,Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Produit,An,Code Produit,,,,,,,,,,,,,,,,,,,,,
Graines Colza/Moutarde,2013,2558,128.424528,4380.000000,0.365000,1.173772e+10,1.230808e+05,1.773585,0.028302,44459.566038,57.811321,2.962264,...,0.028302,586.792453,201.811321,198.415094,2.716981,15.056604,615.066038,6.084906,504.669811,-24.867925
Soja,2013,2555,130.431818,1890.436732,0.272457,2.730573e+11,2.609063e+07,7.174242,0.718712,48838.196970,131.780303,4.833333,...,0.311364,2017.719697,809.295455,765.553030,80.674242,29.818182,2106.765152,55.924242,1714.712121,-45.272727
Arachides Decortiquees,2013,2556,127.545455,5528.955979,0.250818,3.928875e+11,1.702143e+07,21.800000,0.951939,41764.563636,0.006061,7.666667,...,1.811273,179.769697,16.490909,14.884848,73.957576,7.915152,193.260606,7.842424,82.375758,-11.909091
Pois,2013,2547,129.054422,3194.267898,0.224493,1.320856e+11,8.830533e+06,9.312925,0.611156,44546.367347,23.054422,0.190476,...,0.045578,71.496599,31.802721,28.959184,39.653061,2.863946,78.653061,5.700680,0.000000,-4.251701
Légumineuses Autres,2013,2549,126.331250,3212.042409,0.223418,5.893576e+11,3.594262e+07,30.668750,1.970125,42421.375000,41.256250,5.100000,...,0.207313,240.168750,32.625000,34.450000,170.087500,11.756250,237.381250,11.981250,0.000000,0.993750
Haricots,2013,2546,126.461538,3334.153907,0.218899,3.793004e+11,2.408843e+07,28.576923,1.841795,43118.666667,17.128205,0.378205,...,0.129872,144.615385,25.096154,21.923077,111.634615,7.961538,146.083333,7.865385,0.000000,1.737179
"Plantes Oleiferes, Autre",2013,2570,130.658730,3147.705981,0.187886,2.710465e+10,1.614539e+06,2.198413,0.103413,52205.317460,14.079365,10.452381,...,0.169683,71.849206,50.825397,26.936508,7.857143,3.301587,97.706349,1.801587,66.809524,-2.000000
Sésame,2013,2561,131.725000,5389.949980,0.187408,6.683589e+10,2.093688e+06,6.075000,0.188917,56014.866667,0.025000,0.525000,...,0.532417,35.358333,11.083333,11.975000,11.966667,1.400000,38.616667,0.641667,20.766667,-4.141667
"Viande, Autre",2013,2735,128.589744,1118.048458,0.178138,5.152241e+10,7.767502e+06,6.762821,0.952756,44053.493590,0.358974,0.096154,...,0.295192,43.641026,4.923077,7.794872,43.737179,0.083333,40.756410,0.000000,0.108974,0.019231


In [8]:
#   ----- QUESTION 6 -----

# Résultats des questions 6 à 14 pour l'année 2013 
#     ainsi que pour la dernière année disponible dans les données de la FAO.


# Calculez, pour les produits végétaux uniquement, la 'Disponibilité intérieure' mondiale exprimée en kcal sur l'année 2013

# 1000*1000 car la 'Disponibilité intérieure' est exprimée en Milliers de tonnes
data['dispo_int_kprot'] = data['Disponibilité intérieure'] * data['ratio_prot']*10**6
data['dispo_int_kcal'] = data['Disponibilité intérieure'] * data['ratio_kcal_kg']*10**6

data6 = data[(data.origin == 'vegetal') & (data.An == 2013)]

# Total ordre 10**6 en kcak
#             10**6 en prot

# Pour mieux présenter la colonne 'ratio_kcal_kg'
data6 = data6.reindex(columns = ['Code Pays', 'Pays', 'Code Produit', 'Produit', 'An', 'dispo_int_kcal', 'dispo_int_kprot', 'ratio_kcal_kg', 'ratio_prot', 'dispo_k_cal', 'dispo_kg_prot', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'origin', 'Population', 'Aliments pour animaux', 'Autres Utilisations', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 'Semences', 'Traitement', 'Variation de stock'])

# Somme de la disponibilité intérieure mondiale végétale par produit en kcal
data6.groupby(['Produit','An','Code Produit']).sum()


,,,Code Pays,dispo_int_kcal,dispo_int_kprot,ratio_kcal_kg,ratio_prot,dispo_k_cal,dispo_kg_prot,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Population,...,Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Produit,An,Code Produit,,,,,,,,,,,,,,,,,,,,,
"Agrumes, Autres",2013,2614,17971,2.663258e+12,5.679496e+07,1.000954e+04,0.206094,2.479442e+12,5.225693e+07,126.0,2.16,6135991,...,0.77,12177.0,831.0,1202.0,11360.0,810.0,11758.0,0.0,72.0,46.0
"Alcool, non Comestible",2013,2659,15635,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.0,0.00,6352762,...,0.00,21740.0,9900.0,9927.0,0.0,0.0,20763.0,0.0,0.0,950.0
Aliments pour enfants,2013,2680,20312,2.778118e+12,1.173044e+08,4.132341e+05,20.074121,2.763215e+12,1.168921e+08,604.0,25.17,6833369,...,4.68,86.0,1013.0,1099.0,762.0,1.0,0.0,0.0,0.0,8.0
Ananas,2013,2618,20783,7.806676e+12,6.030501e+07,5.450590e+04,0.324196,6.963713e+12,5.419632e+07,640.0,4.58,6810370,...,1.81,23282.0,7671.0,6323.0,20757.0,2745.0,24274.0,0.0,8.0,349.0
Arachides Decortiquees,2013,2556,21244,1.560271e+14,6.758673e+09,8.956909e+05,40.632519,6.482644e+13,2.808536e+09,3597.0,157.07,6934565,...,298.88,29694.0,2721.0,2456.0,12203.0,1307.0,31919.0,1295.0,13620.0,-1965.0
Avoine,2013,2516,17641,4.907772e+13,2.001086e+09,2.306726e+05,9.991921,7.337340e+12,3.212602e+08,854.0,34.72,6200640,...,14.91,23407.0,3532.0,3708.0,3904.0,514.0,23816.0,2485.0,36.0,-577.0
Bananes,2013,2615,20699,6.458100e+13,8.525494e+08,9.560873e+04,1.301365,5.332078e+13,7.021678e+08,3924.0,53.19,6691919,...,12.55,103991.0,21662.0,19423.0,85953.0,15295.0,105747.0,0.0,1739.0,481.0
Bananes plantains,2013,2616,11419,2.522785e+13,2.148391e+08,6.518978e+04,0.593584,1.966419e+13,1.683820e+08,2713.0,23.72,2450042,...,6.22,29233.0,671.0,1124.0,22863.0,3222.0,28778.0,0.0,2410.0,0.0
Bière,2013,2656,21007,8.504023e+13,8.180781e+08,7.296185e+04,0.749123,8.494469e+13,8.171149e+08,6573.0,65.21,6849717,...,0.00,187348.0,13798.0,12565.0,187051.0,43.0,188586.0,0.0,0.0,-2.0


In [9]:
#   ----- QUESTION 7 -----

# Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée 
#  pour de la nourriture ?     Donnez les résultats en termes de calories, puis de protéines, 
#      et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

# Nourriture =  (Disponibilité int.) - (Aliments pour animaux) - (Semences) - (Pertes) - (Traitement) - (Autres Utilisations)

# data6['dispo_int_kcal'] représente la 'Disponibilité intérieure' mondiale en kcal pour chaque produit végétal pour en 2013.
# La somme de cette colonne représente la  'Disponibilité intérieure' mondiale tout produit végétal confondu

# http://www.fao.org/fileadmin/templates/ess/documents/food_security_statistics/MinimumDietaryEnergyRequirement_fr.xls
# Besoin en énergie alimentaire minimum en kcal/personne/jour = 1850 kcal/personne/jour  selon Loic
# Besoin en énergie alimentaire minimum en kcal/personne/jour = 2500 kcal/personne/jour  selon Lou-Andréa

# https://www.ocl-journal.org/articles/ocl/pdf/2003/01/ocl2003101p61.pdf
# Besoin en protéines minimum en gramme/personne/jour = 48 gr pour une personne de 60 kg
# Chapitre :  Apports conseillés en protéines et acides aminés indispensables
# 0.8 gr/kg => pour une personne moyenne de 60kg => donc 48 grammes/personne/jour


# Disponibilité intérieur (kcal et kprot) mondiale vegetale tout Pays et Produit confondus pour l'année 2013 :
dispo_int_veg_monde_kcal = data6.dispo_int_kcal.sum()
dispo_int_veg_monde_kprot = data6.dispo_int_kprot.sum()

dispo_int_veg_monde_kcal = dispo_int_veg_monde_kcal/365         # Résultat par Jour
nb_theo_humain1  = dispo_int_veg_monde_kcal/1850
nb_theo_humain11 = dispo_int_veg_monde_kcal/2500

dispo_int_veg_monde_kprot = dispo_int_veg_monde_kprot/365       # Résultat par Jour
nb_theo_humain2 = dispo_int_veg_monde_kprot/0.048

#print("Population mondiale en 2013 = ", NbHumainTotal)
#print("Nombre d'humains nourris si min_kcal= 1850  : ", int(nb_theo_humain1))
print("Nombre d'humains nourris si min_kcal= 2500  : ", int(nb_theo_humain11))
print(" ")
#print("Nombre d'humains nourris selon kprot = ", int(nb_theo_humain2))

# Pourcentage
print("Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture alors,")
print(int(nb_theo_humain11), " d'humains pourraient être nourris (min_kcal= 2500), soit ",int(nb_theo_humain11*100/NbHumainTotal)," % de la population mondiale" )



Nombre d'humains nourris si min_kcal= 2500  :  13326688493
 
Si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture alors,
13326688493  d'humains pourraient être nourris (min_kcal= 2500), soit  190  % de la population mondiale


In [10]:
#  ----- QUESTION 8 -----

### Combien d'humains pourraient être nourris si :
###   . toute la disponibilité alimentaire en produits végétaux (Food) +
#   . la nourriture végétale destinée aux animaux (Feed) +
#   . les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture ? 
#
# Disponibilité int. = (Production - Quantité) + (Importations - Quantit) - (Exportations - Quantit) + (Variation de stock)
# Nourriture         = (Disponibilité int.) - (Aliments pour animaux) - (Semences) - (Pertes) - (Traitement)  - (Autres Utilisations)
#
# Nourriture (food) + Aliments pour animaux (Feed) + Pertes (Waste)     <-- Végétaux en Milliers de tonnes
# 'Nourriture'      + 'Aliments pour animaux'      + 'Pertes'

#    *10**6 car les valeurs sont exprimées en Milliers de tonnes
# ffw = Food+Feed+Waste
data['dispo_ffw_kcal'] = (data['Nourriture'] * data['ratio_kcal_kg']*10**6) + \
                          (data['Aliments pour animaux'] * data['ratio_kcal_kg']*10**6) + \
                          (data['Pertes'] * data['ratio_kcal_kg']*10**6)
data['dispo_ffw_kprot'] = (data['Nourriture'] * data['ratio_prot']*10**6) + \
                           (data['Aliments pour animaux'] * data['ratio_prot']*10**6) + \
                           (data['Pertes'] * data['ratio_prot']*10**6)

# Calculez, pour les produits végétaux uniquement, la 'Disponibilité intérieure' mondiale exprimée en kcal sur l'année 2013
data8 = data[(data.origin == 'vegetal') & (data.An == 2013)]

# Disponibilité intérieur (kcal et kprot) mondiale vegetale tout Pays et Produit confondus pour l'année 2013 :
dispo_ffw_veg_monde_kcal = data8.dispo_ffw_kcal.sum()
dispo_ffw_veg_monde_kprot = data8.dispo_ffw_kprot.sum()

# Nombre d'humains nourris si toute la dispo intérieure mondiale de produits végétaux était utilisée pour de la nourriture 
dispo_ffw_veg_monde_kcal = dispo_ffw_veg_monde_kcal/365         # Résultat par Jour
nb_theo_humain1 = dispo_ffw_veg_monde_kcal/1850
nb_theo_humain11 = dispo_ffw_veg_monde_kcal/2500

dispo_ffw_veg_monde_kprot = dispo_ffw_veg_monde_kprot/365       # Résultat par Jour
nb_theo_humain2 = dispo_ffw_veg_monde_kprot/0.048

#print("Population mondiale en 2013 = ", NbHumainTotal)
#print("Nombre d'humains nourris si min_kcal= 1850  : ", int(nb_theo_humain1))
print("Nombre d'humains nourris si min_kcal= 2500  : ", int(nb_theo_humain11))
print(" ")
#print("Nombre d'humains nourris selon kprot = ", int(nb_theo_humain2))

# Pourcentage
print("Si toute la disponibilité intérieure mondiale de produits végétaux (Nourriture, Aliments pour animaux, Pertes) était utilisée pour de la nourriture alors,")
print(int(nb_theo_humain11), " d'humains pourraient être nourris (min_kcal= 2500), soit ",int(nb_theo_humain11*100/NbHumainTotal)," % de la population mondiale" )


Nombre d'humains nourris si min_kcal= 2500  :  10047038044
 
Si toute la disponibilité intérieure mondiale de produits végétaux (Nourriture, Aliments pour animaux, Pertes) était utilisée pour de la nourriture alors,
10047038044  d'humains pourraient être nourris (min_kcal= 2500), soit  143  % de la population mondiale


In [11]:
#   ----- QUESTION 9 -----

# Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale = Nourriture Végétale et Animale
# 

#    *10**6 car les valeurs sont exprimées en Milliers de tonnes
# ffw = Food+Feed+Waste
data['dispo_alim_kcal'] = data['Nourriture'] * data['ratio_kcal_kg']*10**6
data['dispo_alim_kprot'] = data['Nourriture'] * data['ratio_prot']*10**6

data9 = data[data['An'] == 2013]
#data9 = data[(data.origin == 'vegetal') & (data.An == 2013)]  # Test : que les végétaux


# Disponibilité intérieur (kcal et kprot) mondiale vegetale tout Pays et Produit confondus pour l'année 2013 :
dispo_alim_monde_kcal = data9.dispo_alim_kcal.sum()
dispo_alim_monde_kprot = data9.dispo_alim_kprot.sum()

# Nombre d'humains nourris si toute la dispo intérieure mondiale de produits végétaux était utilisée pour de la nourriture 
dispo_alim_monde_kcal = dispo_alim_monde_kcal/365            # Résultat par Jour
nb_theo_humain1 = dispo_alim_monde_kcal/1850
nb_theo_humain11 = dispo_alim_monde_kcal/2500

dispo_alim_monde_kprot = dispo_alim_monde_kprot/365          # Résultat par Jour
nb_theo_humain2 = dispo_alim_monde_kprot/0.048

    
#print("Population mondiale en 2013 = ", NbHumainTotal)
#print("Nombre d'humains nourris si min_kcal= 1850  : ", int(nb_theo_humain1))
print("Nombre d'humains nourris si min_kcal= 2500  : ", int(nb_theo_humain11))
print(" ")
#print("Nombre d'humains nourris selon kprot = ", int(nb_theo_humain2))

# Pourcentage
print("Si toute la disponibilité intérieure mondiale de produits végétaux et animaux était répartie alors,")
print(int(nb_theo_humain11), " d'humains pourraient être nourris(min_kcal= 2500), soit ",int(nb_theo_humain11*100/NbHumainTotal)," % de la population mondiale" )


Nombre d'humains nourris si min_kcal= 2500  :  8064590067
 
Si toute la disponibilité intérieure mondiale de produits végétaux et animaux était répartie alors,
8064590067  d'humains pourraient être nourris(min_kcal= 2500), soit  115  % de la population mondiale


In [12]:
#   ----- QUESTION 10 -----

# A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : 
#   Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?
        
# Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.

# Repérez dans vos données les informations concernant les céréales 
#       (par exemple en créant une colonne de type booléen nommée "is_cereal").

# FAO >> Sécurité alimentaire >> Données de la sécurité alimentaire .
#  Cette rubrique contient de nombreux indicateurs, 
#       mais nous ne nous intéresserons ici qu’à l’indicateur  --> "Nombre de personnes sous-alimentées"

# Donnez les résultats des questions 6 à 14 pour l'année 2013 ainsi que pour la dernière année disponible dans les données de la FAO.

# Issu des Données de la sécurité alimentaire de la FAO avec Régions=Monde +(Total) & sélection de toutes les années
data10 = pd.read_csv("Question_10.csv", sep=",")

# Nombre d'humains sous-nourris en 2013 *10**6 (car exprimé en Millions)
nb_humain_sn_2013 = data10[data10.Année == '2013'].Valeur*10**6
print(int(nb_humain_sn_2013), " Humains en sous-nutrition en 2013, soit ",int(nb_humain_sn_2013*100/NbHumainTotal)," % de la population mondiale" )

# Nombre d'humains sous-nourris en 2017
nb_humain_sn_2017 = data10[data10.Année == '2017'].Valeur*10**6
print(int(nb_humain_sn_2017), " Humains en sous-nutrition en 2017, soit ",int(nb_humain_sn_2017*100/NbHumainTotal)," % de la population mondiale" )

794900000  Humains en sous-nutrition en 2013, soit  11  % de la population mondiale
820800000  Humains en sous-nutrition en 2017, soit  11  % de la population mondiale


In [13]:
# Nombre d'humains sous-nourris en 2013 et 2016   *10**6 (car exprimé en Millions)
# secu provient de secu_alimentaire = pd.read_csv("secu_alimentaire.csv")  -->  Voir la 2ème cellule
#
# Issu des Données de la sécurité alimentaire de la FAO avec Pays=tous & Accés>(Liste) &  Nombre de personnes sous-alimentées (millions)
#   sélection de toutes les années

secu13 = secu[secu['An'] == 2013]
secu16 = secu[secu['An'] == 2016]
nb_humain_sn_2013 = secu13.nb_sous_aliment.sum()*10**6   # Millions personnes  (moy 3ans)
nb_humain_sn_2016 = secu16.nb_sous_aliment.sum()*10**6   # Millions personnes  (moy 3ans)

print(int(nb_humain_sn_2013), " Humains en sous-nutrition en 2013, soit ",int(nb_humain_sn_2013*100/NbHumainTotal)," % de la population mondiale" )
print(int(nb_humain_sn_2016), " Humains en sous-nutrition en 2016, soit ",int(nb_humain_sn_2016*100/NbHumainTotal)," % de la population mondiale" )
# 2015=2016=2017 car c'est une moyenne sur 3 ans


782300000  Humains en sous-nutrition en 2013, soit  11  % de la population mondiale
785700000  Humains en sous-nutrition en 2016, soit  11  % de la population mondiale


In [14]:
#   ----- QUESTION 11 -----

# En ne prenant en compte que les céréales (data.is_cereal = True) destinées à l'alimentation (humaine et animale ===
#    ===> 'Nourriture' + 'Aliments pour animaux'.
#       quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

# WHEAT     = BLÉ    2511
# RICE      = RIZ    2805
# BARLEY    = ORGE   2513
# MAIZE     = MAÏS   2514
# RYE       = SEIGLE 2515
# OATS      = AVOINE 2516
# MILLET    = MILLET 2517
# SORGHUM   = SORGHO 2518
# BUCKWHEAT = SARRASIN  Céréales autres	2520
# QUINOA    = QUINOA    Céréales autres
# FONIO     = FONIO     Céréales autres
# TRITICALE = TRITICALE Céréales autres

data['is_cereal'] = False

list_des_codes = [2511,2805,2513,2514,2515,2516,2517,2518,2520]
data["is_cereal"] = data['Code Produit'].isin(list_des_codes)

data11 = data[(data.is_cereal == 1) & (data.An == 2013)]

Alim_Humaine = int(data11['Nourriture'].sum())             # Céréales pour les humains
Alim_Animale = int(data11['Aliments pour animaux'].sum())  # Céréales pour les animaux
Alim_Totale  = Alim_Humaine + Alim_Animale
Proportion   = Alim_Animale/Alim_Totale

print("Alimentation Humaine = ", Alim_Humaine, " Milliers de Tonnes")
print("Alimentation Animale = ", Alim_Animale, " Milliers de Tonnes")
print(Proportion*100, "%  de la production céréaliaires est destinée aux animaux.")

Alimentation Humaine =  1029012  Milliers de Tonnes
Alimentation Animale =  873536  Milliers de Tonnes
45.91400584899829 %  de la production céréaliaires est destinée aux animaux.


In [15]:
#   ----- QUESTION 12 ----

# Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)
#
# - le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
# - le ratio entre la quanFtité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)
#
# Page de référence des pays en sous-nutrition :  
#        https://fr.wikipedia.org/wiki/Liste_des_pays_par_proportion_de_personnes_sous-aliment%C3%A9es
#
# 'Importations - Quantité' en Milliers de tonnes
# Certains Pays de cette page n'ont pas été pris en compte car le FAO ne les référence pas
#
# - le ratio entre la quantité destinés aux "Autres utilisations" et la "Disponibilité intérieure".
# - le ratio entre la quantité destinée à la nourriture animale ("Aliments pour animaux")
#                             et la quantité destinée à la "Nourriture" (animale + humaine)

data['is_sous_nutri'] = False

# Liste des Pays en sous nutrition
list_des_pays_sn = ["Zambie","Haïti","Éthiopie","Soudan","Mozambique","Timor-Leste","Congo","Afghanistan","Ouganda", \
"Namibie","Tchad","Madagascar","Zimbabwe","Yémen","Tadjikistan","République populaire démocratique de Corée","Libéria","Kenya", \
"Guatemala","Rwanda","Sierra Leone","Botswana","Angola","Iraq","Burkina Faso","Paraguay","Géorgie","Mongolie", \
"Bolivie (État plurinational de)","Sri Lanka","Malawi","Sénégal","Nicaragua","Pakistan","Djibouti", "Myanmar","Équateur", \
"Grenade","Inde","Guinée","Cambodge","Philippines","Bangladesh","Lesotho","Togo","Cameroun","République dominicaine", \
"Sainte-Lucie","Gambie","Îles Salomon","Niger","Colombie","El Salvador","Chine, continentale","Suriname","Pérou","Panama", \
"Honduras","Trinité-et-Tobago","Mauritanie","Viet Nam","Guinée-Bissau","Jamaïque","Indonésie","Seychelles","Nigéria","Vanuatu", \
"Bénin","Mali","Sao Tomé-et-Principe","Thaïlande","Bahamas","Brésil","Belize","Gabon","Costa Rica","Kirghizistan","Ouzbékistan", \
"Maurice","Maldives","Maroc","Guyana","Saint-Vincent-et-les Grenadines","Ghana","Nouvelle-Calédonie", \
"Iran (République islamique d')","Arménie","Venezuela (République bolivarienne du)","Azerbaïdjan","Turkménistan", \
"Kazakhstan","Jordanie","Algérie","Samoa","Koweït","Cuba","Chili","Uruguay","Fidji"]

data["is_sous_nutri"] = data['Pays'].isin(list_des_pays_sn)


data12 = data[(data.is_sous_nutri == 1) & (data.An == 2013)]
# Rmque :  'is_sous_nutri' peut être mis en place aussi avec le dataframe secu, 
#                           qui contient la liste des pays en sous-nutrition


data12 = data12.sort_values(by = 'Exportations - Quantité', ascending = False)

# Pour mieux présenter les colonnes 'Exportations - Quantité' et 'Importations - Quantité'
data12 = data12.reindex(columns = ['Produit', 'Pays', 'An', 'Production', 'Exportations - Quantité', \
        'Importations - Quantité', \
       'origin', 'Code Pays', 'Code Produit','Population', 'Aliments pour animaux', 'Autres Utilisations', \
       'Disponibilité alimentaire (Kcal/personne/jour)', \
       'Disponibilité alimentaire en quantité (kg/personne/an)', \
       'Disponibilité de matière grasse en quantité (g/personne/jour)', \
       'Disponibilité de protéines en quantité (g/personne/jour)', \
       'Disponibilité intérieure',  'Nourriture', 'Pertes', \
       'Semences', 'Traitement', 'Variation de stock', 'dispo_k_cal', 'dispo_kg_prot', 'ratio_kcal_kg', 'ratio_prot', \
        'is_sous_nutri'])


# Pour la Question 14
data14 = data12[data12.Pays == 'Thaïlande']
data14 = data14[data14.Produit == 'Manioc']

# Pour la Question 13
data13 = data12[data12.Pays == "États-Unis d'Amérique"]
data13 = data13[data13.origin == 'animal']


# 15 produits les plus exportés par ce groupe de pays sur l'année choisie - 2013.
a = data12.Produit.unique()[0:15]

# 'Soja', 'Sucre Eq Brut', 'Maïs', 'Manioc', 'Huile de Palme','Oranges, Mandarines', 'Riz (Eq Blanchi)', 'Légumes, Autres',
# 'Blé', 'Bananes', 'Poissons Pelagiques', 'Tomates', 'Viande de Volailles', 'Pommes', 'Fruits, Autres'
data12 = data12[data12.Produit.isin(a)]   # selection de ces 15 produits

# Sélectionnez les 200 plus grandes importations de ces 15 produits,
data12.sort_values(by = 'Importations - Quantité', ascending = False, inplace=True)
data12 = data12.iloc[0:200]

# Importations groupées par produit
data12 = data12.groupby(['Produit']).sum()
data12.sort_values(by = 'Importations - Quantité', ascending = False, inplace=True)
data12.head(10)


,An,Production,Exportations - Quantité,Importations - Quantité,Code Pays,Code Produit,Population,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),...,Nourriture,Pertes,Semences,Traitement,Variation de stock,dispo_k_cal,dispo_kg_prot,ratio_kcal_kg,ratio_prot,is_sous_nutri
Produit,,,,,,,,,,,,,,,,,,,,,
Blé,106689,79527.0,5163.0,77758.0,6123,133083,2167522,14877.0,5465.0,29710.0,...,118661.0,6777.0,4236.0,765.0,-1386.0,3.379422e+14,9.614156e+09,150181.127223,4.330255,53.0
Maïs,46299,128339.0,27159.0,31396.0,2397,57822,1200895,82369.0,5704.0,6126.0,...,28963.0,11579.0,874.0,924.0,-2167.0,8.205762e+13,1.988533e+09,71347.953664,1.794831,23.0
Sucre Eq Brut,54351,39332.0,4318.0,21542.0,3303,68634,2612420,20.0,2845.0,5834.0,...,48650.0,17.0,0.0,19.0,-4244.0,1.725272e+14,0.000000e+00,96209.635961,0.000000,27.0
Lait - Excl Beurre,48312,104186.0,1565.0,19682.0,2876,68352,1549284,6083.0,559.0,3470.0,...,109300.0,6430.0,0.0,0.0,33.0,6.626855e+13,3.758115e+09,13704.660296,0.804262,24.0
Huile de Palme,30195,1350.0,457.0,17905.0,1837,38655,2443792,0.0,12153.0,1281.0,...,5803.0,58.0,0.0,0.0,-786.0,4.961430e+13,7.201235e+06,113317.234977,0.010735,15.0
Riz (Eq Blanchi),48312,114554.0,1073.0,16273.0,2856,67320,1310536,4353.0,8224.0,13513.0,...,99060.0,8583.0,1869.0,737.0,-6928.0,3.585055e+14,6.740412e+09,86452.781571,1.675905,24.0
Soja,16104,83127.0,42830.0,5398.0,1005,20440,769964,732.0,194.0,134.0,...,1725.0,262.0,1273.0,40307.0,-1203.0,5.064937e+12,5.057635e+08,19640.946215,1.989990,8.0
Poissons Pelagiques,14091,5741.0,3596.0,4940.0,699,19341,598076,1829.0,0.0,142.0,...,5257.0,0.0,0.0,0.0,0.0,4.723626e+12,7.016639e+08,6645.818119,0.916533,7.0
"Fruits, Autres",16104,24054.0,4024.0,2909.0,1116,21000,709437,0.0,0.0,255.0,...,20578.0,2887.0,0.0,6.0,58.0,8.363851e+12,1.013561e+08,3043.172218,0.036659,8.0


In [22]:
#   ----- QUESTION 12 ----

# 3 meilleurs ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure
data12['ratio_autres_util'] = data12['Autres Utilisations'] / data12['Disponibilité intérieure']
data12.sort_values(by = 'ratio_autres_util', ascending = False, inplace=True)
data12.head(3)
# Réponse :  Alcool non Comestible, Huile de Palme, Manioc.

,An,Production,Exportations - Quantité,Importations - Quantité,Code Pays,Code Produit,Population,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),...,Semences,Traitement,Variation de stock,dispo_k_cal,dispo_kg_prot,ratio_kcal_kg,ratio_prot,is_sous_nutri,ratio_autres_util,ratio_animal_humain
Produit,,,,,,,,,,,,,,,,,,,,,
"Alcool, non Comestible",4026,69.0,96.0,557.0,280,5318,101178,0.0,530.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.00,0.000000,0.000000,2.0,1.000000,NaN
Huile de Palme,30195,1350.0,457.0,17905.0,1837,38655,2443792,0.0,12153.0,1281.0,...,0.0,0.0,-786.0,4.961430e+13,7201234.65,113317.234977,0.010735,15.0,0.674679,0.000000
Manioc,6039,56526.0,26038.0,2628.0,488,7596,415271,2422.0,11176.0,236.0,...,0.0,0.0,0.0,1.531539e+13,83067101.50,3152.705688,0.016272,3.0,0.337491,0.162605


In [24]:
#   ----- QUESTION 12 ----

# 3 meilleurs ratios entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale+humaine)
data12['ratio_animal_humain'] = data12['Aliments pour animaux'] / data12['Nourriture']
data12.sort_values(by = 'ratio_animal_humain', ascending = False, inplace=True)
data12.head(3)
# Réponse :  Maïs, Soja, Poissons Pelagiques

,An,Production,Exportations - Quantité,Importations - Quantité,Code Pays,Code Produit,Population,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),...,Semences,Traitement,Variation de stock,dispo_k_cal,dispo_kg_prot,ratio_kcal_kg,ratio_prot,is_sous_nutri,ratio_autres_util,ratio_animal_humain
Produit,,,,,,,,,,,,,,,,,,,,,
Maïs,46299,128339.0,27159.0,31396.0,2397,57822,1200895,82369.0,5704.0,6126.0,...,874.0,924.0,-2167.0,8.205762e+13,1.988533e+09,71347.953664,1.794831,23.0,0.043739,2.843939
Soja,16104,83127.0,42830.0,5398.0,1005,20440,769964,732.0,194.0,134.0,...,1273.0,40307.0,-1203.0,5.064937e+12,5.057635e+08,19640.946215,1.989990,8.0,0.004360,0.424348
Poissons Pelagiques,14091,5741.0,3596.0,4940.0,699,19341,598076,1829.0,0.0,142.0,...,0.0,0.0,0.0,4.723626e+12,7.016639e+08,6645.818119,0.916533,7.0,0.000000,0.347917


In [18]:
#   ----- QUESTION 13 ----

# Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

# Production de produits animaux des USA en 2013
data13 = data[(data.Pays == "États-Unis d'Amérique") & (data.An == 2013) & (data.origin == 'animal')]
prod_usa_ani = data13.Production.sum()

# Consommation de Céréales comme 'Aliments pour animaux des USA en 2013
# Utilisation de data11 car -> data11 = data[(data.is_cereal == 1) & (data.An == 2013)]
data11 = data11[data11.Pays == "États-Unis d'Amérique"]
Conso_cereal_par_Animal = int(data11['Aliments pour animaux'].sum())

print("Production de produits animaux des USA en 2013 (Milliers de Tonnes) = ", prod_usa_ani)
print("Consommation de Céréales comme 'Aliments pour animaux des USA en 2013 (Milliers de Tonnes) = ", Conso_cereal_par_Animal)
print("Aux USA, une diminution de 10% de la production de produits animaux, représentent un gain de : ", Conso_cereal_par_Animal/10, " Milliers de Tonnes de céréales")


Production de produits animaux des USA en 2013 (Milliers de Tonnes) =  154474.0
Consommation de Céréales comme 'Aliments pour animaux des USA en 2013 (Milliers de Tonnes) =  140096
Aux USA, une diminution de 10% de la production de produits animaux, représentent un gain de :  14009.6  Milliers de Tonnes de céréales


In [19]:
#   ----- QUESTION 14 ----

# En Thaïlande, quel proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?
#  = 7.3 % de personne en sous-nutrition
manioc_export = data14['Exportations - Quantité'] / data14['Production']
print("Proportion de Manioc exporté de Thaïlande = ", int(manioc_export*100), "%" )

pop_thailande_2013 = int(data14['Population'])*10**3
pop_thailande_sn_2013 = secu[(secu.Pays == 'Thaïlande') & (secu.An == 2013)]['nb_sous_aliment']*10**6
pop_thailande_sn_2016 = secu[(secu.Pays == 'Thaïlande') & (secu.An == 2016)]['nb_sous_aliment']*10**6
#pop_thailande_sn_2013 = int(secu[(secu.Pays == 'Thaïlande') & (secu.An == 2013)]['nb_sous_aliment'])*10**6
#pop_thailande_sn_2016 = int(secu[(secu.Pays == 'Thaïlande') & (secu.An == 2016)]['nb_sous_aliment'])*10**6

Proportion2013   = pop_thailande_sn_2013/pop_thailande_2013
Proportion2016   = pop_thailande_sn_2016/pop_thailande_2013


print("En Thaîlande 2013, ", Proportion2013*100, "%  des habitants sons en sous nutrition.")
print("En Thaîlande 2016, ", Proportion2016*100, "%  des habitants sons en sous nutrition.")

# Résultats des questions 6 à 14 pour l'année 2013 
#     ainsi que pour la dernière année disponible dans les données de la FAO.


Proportion de Manioc exporté de Thaïlande =  83 %
En Thaîlande 2013,  686    9.102983
Name: nb_sous_aliment, dtype: float64 %  des habitants sons en sous nutrition.
En Thaîlande 2016,  689    9.252212
Name: nb_sous_aliment, dtype: float64 %  des habitants sons en sous nutrition.
